In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load datasets
bird_stories = pd.read_csv('/content/drive/MyDrive/data/BARD/BARD_story.csv')  # columns: index, prompts, BARD
gpt_stories = pd.read_csv('/content/drive/MyDrive/data/GPT/ChatGPT_story.csv')  # columns: index, prompts, responses
human_stories = pd.read_csv('/content/drive/MyDrive/data/Human/human_stories.csv')  # columns: index, Title, Author, Tags, Chapter_text

# Prepare the texts and add source labels
bird_stories['text'] = bird_stories['BARD']
gpt_stories['text'] = gpt_stories['Chapter_text']
human_stories['text'] = human_stories['Chapter_text']

# Add source labels
bird_stories['source'] = 1  # BARD
gpt_stories['source'] = 0    # GPT
human_stories['source'] = 2   # Human

# Combine datasets
stories_data = pd.concat([bird_stories[['text', 'source']], gpt_stories[['text', 'source']], human_stories[['text', 'source']]], ignore_index=True)

# Clean the text
stories_data['text'] = (
    stories_data['text'].str.replace(r'http\S+|www\S+|https\S+', '', case=False)  # Remove URLs
    .str.replace(r'<.*?>', '', case=False)  # Remove HTML tags
    .str.replace(r'\s+', ' ', regex=True)  # Remove extra whitespaces
    .str.lower()  # Convert to lowercase
)

# Handle NaN values: replace NaN with an empty string
stories_data['text'] = stories_data['text'].fillna('')

# Prepare features and labels
X = stories_data['text']
y = stories_data['source']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Calculate max sequence length and vocabulary size
max_length = max(len(seq) for seq in X_sequences)
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding

# Pad sequences
X_padded = pad_sequences(X_sequences, maxlen=max_length)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42, stratify=y)

# GRU model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(GRU(64, return_sequences=False))  # GRU layer for sequential data
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(64, activation='relu'))  # Fully connected layer
model.add(Dense(3, activation='softmax'))  # 3 classes for multiclass classification

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(classification_report(y_test, y_pred, target_names=['GPT', 'BARD', 'Human']))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 306s 19s/step - accuracy: 0.4535 - loss: 1.0774 - val_accuracy: 0.5217 - val_loss: 0.9821
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 312s 18s/step - accuracy: 0.5310 - loss: 0.9211 - val_accuracy: 0.5391 - val_loss: 0.7800
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 330s 19s/step - accuracy: 0.6731 - loss: 0.6390 - val_accuracy: 0.8087 - val_loss: 0.4653
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 284s 19s/step - accuracy: 0.8144 - loss: 0.3895 - val_accuracy: 0.8261 - val_loss: 0.3438
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 327s 19s/step - accuracy: 0.8341 - loss: 0.3855 - val_accuracy: 0.8348 - val_loss: 0.3681
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 317s 19s/step - accuracy: 0.8384 - loss: 0.2904 - val_accuracy: 0.8348 - val_loss: 0.3775
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 296s 20s/step - accuracy: 0.8363 - loss: 0.2719 - val_accuracy: 0.8348 - val_loss: 0.3725
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 314s 19s/step - accuracy: 0.8778 - loss: 0.2367 - val_accuracy: 0.8609 - val_

3/4 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step
Accuracy: 0.8435
Precision: 0.8557
Recall: 0.8435
F1-score: 0.8142
              precision    recall  f1-score   support

         GPT       0.80      0.21      0.33        19
        BARD       0.97      0.97      0.97        60
       Human       0.70      0.97      0.81        36

    accuracy                           0.84       115
   macro avg       0.82      0.72      0.70       115
weighted avg       0.86      0.84      0.81       115

